# Prediction model with Tensorflow

In [1]:
%pylab inline
import tensorflow as tf
import sys

# Include additional module
include_path = '../tensorflow_oop/'
if include_path not in sys.path:
    sys.path.append(include_path)
from tensorflow_oop.regression import *

Populating the interactive namespace from numpy and matplotlib


## Create dataset

### Load dump of dataset

In [2]:
DUMP_PATH = '../data/moscow.dump'

print('Loading dataset...')
dataset = TFDataset.load(DUMP_PATH)
print('Dataset shape: %s %s -> %s' % (dataset.size_, dataset.data_shape_, dataset.labels_shape_))

Loading dataset...
Dataset shape: 36567 [32, 3] -> [1, 3]


### Set batch size

In [3]:
BATCH_SIZE = 32

dataset.set_batch_size(BATCH_SIZE)

### Split dataset

In [4]:
TRAIN_RATE = 0.70
VAL_RATE = 0.15
TEST_RATE = 0.15
SHUFFLE = False

print('Splitting...')
train_set, val_set, test_set = dataset.split(TRAIN_RATE, VAL_RATE, TEST_RATE, shuffle=SHUFFLE)
print('Traininig  set shape: %s %s -> %s' % (train_set.size_, train_set.data_shape_, train_set.labels_shape_))
print('Validation set shape: %s %s -> %s' % (val_set.size_,   val_set.data_shape_,   val_set.labels_shape_))
print('Testing    set shape: %s %s -> %s' % (test_set.size_,  test_set.data_shape_,  test_set.labels_shape_))

Splitting...
Traininig  set shape: 25597 [32, 3] -> [1, 3]
Validation set shape: 5485 [32, 3] -> [1, 3]
Testing    set shape: 5485 [32, 3] -> [1, 3]


## Define model

In [5]:
class TFWeatherForecast(TFRegressor):
    def inference(self, inputs, **kwargs):
        input_size = self.inputs_shape_[0]
        hidden_size = kwargs['hidden_size']
        rnn_count = kwargs['rnn_count']
        output_size = self.outputs_shape_[-1]
        def rnn_cell():
            return tf.contrib.rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
        cells = [rnn_cell() for _ in range(rnn_count)]
        multi_cell = tf.nn.rnn_cell.MultiRNNCell(cells, state_is_tuple=True)
        batch_size = tf.shape(inputs)[0]
        init_state = multi_cell.zero_state(batch_size, dtype=tf.float32)
        rnn_outputs, last_state = tf.nn.dynamic_rnn(cell=multi_cell,
                                                    initial_state=init_state,
                                                    inputs=inputs,
                                                    time_major=False)
        W = tf.Variable(tf.truncated_normal([hidden_size, output_size]))
        b = tf.Variable(tf.zeros([output_size]))
        outputs = tf.nn.xw_plus_b(last_state[-1].h, W, b)
        return tf.reshape(outputs, [-1] + self.outputs_shape_)

In [6]:
LOG_DIR = '/tmp/prediction-model'
INPUTS_SHAPE = dataset.data_shape_
OUTPUTS_SHAPE = dataset.labels_shape_
HIDDEN_SIZE = 100
RNN_COUNT = 1

model = TFWeatherForecast(log_dir=LOG_DIR,
                          inputs_shape=INPUTS_SHAPE,
                          outputs_shape=OUTPUTS_SHAPE,
                          hidden_size=HIDDEN_SIZE,
                          rnn_count=RNN_COUNT)
print(model)

Start initializing model...
Finish initializing model.
TFNeuralNetwork object:
            log_dir_: /tmp/prediction-model
       inputs_shape_: [32, 3]
      outputs_shape_: [1, 3]
   data_placeholder_: Tensor("input_data:0", shape=(?, 32, 3), dtype=float32)
 labels_placeholder_: Tensor("input_labels:0", shape=(?, 1, 3), dtype=float32)
            outputs_: Tensor("output_layer:0", shape=(?, 1, 3), dtype=float32)
            metrics_: {'loss': <tf.Tensor 'loss:0' shape=() dtype=float32>}
               loss_: Tensor("loss:0", shape=(), dtype=float32)
               sess_: <tensorflow.python.client.session.Session object at 0x11733ae10>
             kwargs_: {'hidden_size': 100, 'rnn_count': 1}
            summary_: Tensor("Merge/MergeSummary:0", shape=(), dtype=string)
     summary_writer_: <tensorflow.python.summary.writer.writer.FileWriter object at 0x1173a0dd8>
   projector_config_: 


## Train model

In [7]:
EPOCH_COUNT = 1

model.fit(train_set, iteration_count=None, epoch_count=EPOCH_COUNT, val_set=val_set)

Start training iteration...
Iteration 100 / 800 (epoch 0 / 1):   loss = 0.006551   [5.535 sec]
Iteration 200 / 800 (epoch 0 / 1):   loss = 0.023786   [5.445 sec]
Iteration 300 / 800 (epoch 0 / 1):   loss = 0.004921   [5.219 sec]
Iteration 400 / 800 (epoch 0 / 1):   loss = 0.012909   [5.158 sec]
Iteration 500 / 800 (epoch 0 / 1):   loss = 0.005228   [5.429 sec]
Iteration 600 / 800 (epoch 0 / 1):   loss = 0.004367   [5.160 sec]
Iteration 700 / 800 (epoch 0 / 1):   loss = 0.003515   [5.265 sec]
Iteration 800 / 800 (epoch 1 / 1):   loss = 0.001730   [5.387 sec]
Saving checkpoint...
INFO:tensorflow:/tmp/prediction-model/fit-checkpoint-800 is not in all_model_checkpoint_paths. Manually adding it.
Model saved to: /tmp/prediction-model/fit-checkpoint-800
Evaluation...
Evaluation on full dataset:   [training   set]   loss = 0.001891   [4.762 sec]
Evaluation on full dataset:   [validation set]   loss = 0.001956   [1.062 sec]
Finish training iteration (total time 60.431 sec).



## Evaluate model

In [8]:
print('Evaluating...')
if train_set is not None:
    train_eval = model.evaluate(train_set)
    print('Results on training   set: %s' % train_eval)
if val_set is not None:
    val_eval = model.evaluate(val_set)
    print('Results on validation set: %s' % val_eval)
if test_set is not None:
    test_eval = model.evaluate(test_set)
    print('Results on testing    set: %s' % test_eval)

Evaluating...
Results on training   set: {'loss': 0.0018911378}
Results on validation set: {'loss': 0.0019556019}
Results on testing    set: {'loss': 0.0016928277}
